In [1]:
import os
import cv2
import numpy as np

# Specify the path to the preprocessed images directory
preprocessed_dir = 'preprocessed_images'

# Specify the path to the output directory for extracted features
output_dir = 'extracted_features2'

# Create the output directory if it doesn't exist
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Iterate over the preprocessed image files
for root, dirs, files in os.walk(preprocessed_dir):
    for file_name in files:
        # Read the preprocessed image
        image_path = os.path.join(root, file_name)
        image = cv2.imread(image_path)

        # Perform feature extraction (e.g., SIFT, SURF, or CNN-based feature extraction)
        # Example using SIFT
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        sift = cv2.SIFT_create()
        keypoints, descriptors = sift.detectAndCompute(gray, None)

        # Save the extracted features
        feature_file = os.path.join(output_dir, f'{file_name}.npy')
        np.save(feature_file, descriptors)

        print(f"Features extracted for {file_name}")

print("Feature extraction completed.")


Features extracted for blurred_IMG_20230410_130327.jpg
Features extracted for blurred_IMG_20230410_130352.jpg
Features extracted for blurred_IMG_20230410_130733.jpg
Features extracted for blurred_IMG_20230410_130823.jpg
Features extracted for blurred_IMG_20230410_130913.jpg
Features extracted for blurred_IMG_20230410_131008.jpg
Features extracted for blurred_IMG_20230410_131353.jpg
Features extracted for blurred_IMG_20230410_132726.jpg
Features extracted for blurred_IMG_20230410_132758.jpg
Features extracted for blurred_IMG_20230410_132823.jpg
Features extracted for blurred_IMG_20230410_134317.jpg
Features extracted for blurred_IMG_20230410_134343.jpg
Features extracted for blurred_IMG_20230410_134455.jpg
Features extracted for blurred_IMG_20230410_134518.jpg
Features extracted for blurred_IMG_20230410_135002.jpg
Features extracted for blurred_IMG_20230410_135018.jpg
Features extracted for blurred_IMG_20230410_135707.jpg
Features extracted for blurred_IMG_20230410_140201_1.jpg
Features

Features extracted for median_filtered_IMG_20230411_154103.jpg
Features extracted for median_filtered_IMG_20230411_154108.jpg
Features extracted for median_filtered_IMG_20230411_154115.jpg
Features extracted for median_filtered_IMG_20230411_154807.jpg
Features extracted for median_filtered_IMG_20230413_130827.jpg
Features extracted for median_filtered_IMG_20230413_130834.jpg
Features extracted for median_filtered_IMG_20230413_130916.jpg
Features extracted for median_filtered_IMG_20230413_131156.jpg
Feature extraction completed.


In [2]:
import tensorflow as tf
print(tf.__version__)


2.12.0


In [12]:
import os

preprocessed_dir = 'preprocessed_images'
labels_file = 'labels.txt'

# Get the unique labels from the image file names
labels = set()
for root, dirs, files in os.walk(preprocessed_dir):
    for file_name in files:
        label = file_name.split('_')[1].split('.')[0]
        labels.add(label)

# Save the labels to the labels file
with open(labels_file, 'w') as f:
    f.write('\n'.join(sorted(labels)))


In [1]:
import os
import zipfile
import imageio
from skimage.filters import gaussian
from scipy.ndimage import median_filter
from skimage.morphology import disk
from skimage.restoration import denoise_nl_means
from skimage.transform import resize


import os
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.utils import to_categorical

# Path to the preprocessed images directory
preprocessed_dir = 'preprocessed_images'

# Path to the labels file
labels_file = 'labels.txt'

# Load the labels
with open(labels_file, 'r') as f:
    labels = f.read().splitlines()

# Create a mapping from labels to numerical values
label_to_index = {label: index for index, label in enumerate(labels)}

# Initialize empty lists to store image data and corresponding labels
images = []
image_labels = []

# Iterate over the preprocessed images directory
for root, dirs, files in os.walk(preprocessed_dir):
    for file_name in files:
        # Load the image
        image_path = os.path.join(root, file_name)
        image = imageio.imread(image_path)
        
        # Append the image data to the list
        images.append(image)
        
        # Extract the label from the file name
        label = file_name.split('_')[1].split('.')[0]
        
        # Map the label to its numerical value
        label_index = label_to_index[label]
        
        # Append the label index to the list
        image_labels.append(label_index)

# Convert the image and label lists to numpy arrays
images = np.array(images)
image_labels = np.array(image_labels)

# Normalize the image data
images = images / 255.0

# Convert the labels to categorical format
num_classes = len(labels)
image_labels = to_categorical(image_labels, num_classes=num_classes)

# Split the dataset into training and testing sets (80% for training, 20% for testing)
split_idx = int(0.8 * len(images))
train_images, test_images = images[:split_idx], images[split_idx:]
train_labels, test_labels = image_labels[:split_idx], image_labels[split_idx:]
image_height, image_width, _ = images[0].shape
# Create the CNN model
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(image_height, image_width, 3)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(train_images, train_labels, epochs=10, batch_size=32, validation_data=(test_images, test_labels))



C:\Users\SK VERMA\AppData\Local\Temp\ipykernel_47528\1664330166.py:39: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  image = imageio.imread(image_path)


Epoch 1/10
4/4 [==============================] - 32s 7s/step - loss: 15.8647 - accuracy: 0.8381 - val_loss: 0.0115 - val_accuracy: 1.0000
Epoch 2/10
4/4 [==============================] - 26s 6s/step - loss: 1.6238 - accuracy: 0.6381 - val_loss: 1.4045 - val_accuracy: 0.0000e+00
Epoch 3/10
4/4 [==============================] - 27s 7s/step - loss: 0.4390 - accuracy: 0.8381 - val_loss: 1.1329 - val_accuracy: 0.0000e+00
Epoch 4/10
4/4 [==============================] - 26s 6s/step - loss: 0.3413 - accuracy: 0.8381 - val_loss: 2.6650 - val_accuracy: 0.0000e+00
Epoch 5/10
4/4 [==============================] - 26s 6s/step - loss: 0.3139 - accuracy: 0.8571 - val_loss: 3.9397 - val_accuracy: 0.0000e+00
Epoch 6/10
4/4 [==============================] - 26s 6s/step - loss: 0.2431 - accuracy: 0.8571 - val_loss: 3.1906 - val_accuracy: 0.0000e+00
Epoch 7/10
4/4 [==============================] - 26s 6s/step - loss: 0.2674 - accuracy: 0.8571 - val_loss: 1.1544 - val_accuracy: 0.4074
Epoch 8/10
4/

In [2]:
# Evaluate the model on the test data
loss, accuracy = model.evaluate(test_images, test_labels)

# Make predictions on the test data
predictions = model.predict(test_images)

# Convert predictions to class labels
predicted_labels = np.argmax(predictions, axis=1)

# Convert true labels to class labels
true_labels = np.argmax(test_labels, axis=1)

# Calculate the metrics
from sklearn.metrics import classification_report

report = classification_report(true_labels, predicted_labels, target_names=labels)
print(report)


1/1 [==============================] - 2s 2s/step
              precision    recall  f1-score   support

         IMG       0.00      0.00      0.00         0
    filtered       1.00      0.44      0.62        27

    accuracy                           0.44        27
   macro avg       0.50      0.22      0.31        27
weighted avg       1.00      0.44      0.62        27



C:\Python310\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Python310\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Python310\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [3]:
# Evaluate the model on the test data
loss, accuracy = model.evaluate(test_images, test_labels)

# Print the accuracy
print('Accuracy:', accuracy)


1/1 [==============================] - 2s 2s/step - loss: 0.7320 - accuracy: 0.4444
Accuracy: 0.4444444477558136
